In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd 
import numpy as np
import os

#import data 
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/raw_data/"
os.getcwd()

Mounted at /content/drive
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [3]:
from os import listdir
FILE_LIST = listdir(DATA_PATH)
print(FILE_LIST)

['con_prod_hh_over_trx_yr_mo_usd.csv', 'con_pur_hh_by_trx_yr_mo_usd.csv', 'demo_hh.csv', 'demo_ind.csv', 'diaries_goingson_hh.csv', 'diaries_wellbeing_ind.csv', 'edu_hh.csv', 'edu_ind.csv', 'housing_conditions_ownership.csv', 'inc_hh_by_trx_yr_mo_usd_incRR.csv', 'inc_ind_by_trx_yr_mo_usd_incRR.csv', 'pov_hh_by_trx_yr_mo_usd_incRR.csv', 'rem_hh_by_trx_yr_mo_usd.csv', 'pov_hh_by_trx_yr_mo_usd_excRR.csv', 'diaries_transactions_all.csv']


In [4]:
FILE_LIST[0].split(".csv")[0]

'con_prod_hh_over_trx_yr_mo_usd'

In [5]:
df_dic={}
for i, e in enumerate(FILE_LIST):
  df_dic[str(FILE_LIST[i].split(".csv")[0])]= pd.read_csv("raw_data/" + e)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df_dic['con_prod_hh_over_trx_yr_mo_usd']



,hh_ids,con_prod_hh_medpti
0,KELDK01,0.000000
1,KELDK02,0.000000
2,KELDK08,0.000000
3,KELDK09,0.000000
4,KELDK10,0.000000
...,...,...
286,KVIHK34,5.421177
287,KVIHK36,0.658824
288,KVIHK38,2.047059
289,KVIHK39,0.000000


In [ ]:
df_dic['transaction']

In [10]:
df_dic['diaries_transactions_all'].merge(df_dic['demo_hh'], how="left", on="hh_ids")



,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_hh_ages15plus,dem_hh_ages18plus,dem_hh_ages65plus,dem_hh_ages20_30,dem_hh_workage,dem_hh_dependents,dem_hh_agehead,dem_hh_malehead,dem_pae_oecd,dem_pae_kihbs
0,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,02oct2013,10,2013,...,2,2,0,2,0.500000,0.500000,27,1,2.7,2.48
1,KVIHC16,NaN,24sep2012,19sep2013,360,12,04=Diaries Interview,22apr2013,4,2013,...,2,1,0,0,0.400000,0.600000,39,0,3.2,3.95
2,KELDK21,NaN,20sep2012,22oct2013,397,13,04=Diaries Interview,02oct2013,10,2013,...,4,4,1,1,0.750000,0.250000,56,1,3.1,4.00
3,KMOMT22,NaN,21sep2012,01nov2013,406,14,04=Diaries Interview,21feb2013,2,2013,...,2,2,0,1,0.285714,0.714286,35,1,4.2,4.43
4,KMOMT32,NaN,20sep2012,18oct2013,393,13,04=Diaries Interview,02may2013,5,2013,...,6,5,0,2,0.600000,0.400000,51,1,6.5,8.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483944,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,02aug2013,8,2013,...,2,2,0,2,0.666667,0.333333,26,1,2.2,2.24
483945,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,04oct2013,10,2013,...,2,2,0,2,0.666667,0.333333,26,1,2.2,2.24
483946,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,21aug2013,8,2013,...,2,2,0,2,0.666667,0.333333,26,1,2.2,2.24
483947,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,01aug2013,8,2013,...,5,4,1,1,0.666667,0.333333,53,1,4.3,5.65


,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr
0,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,02oct2013,10,2013,...,NaN,NaN,NaN,0.0,0.000000,0,NaN,NaN,NaN,13.586210
1,KVIHC16,NaN,24sep2012,19sep2013,360,12,04=Diaries Interview,22apr2013,4,2013,...,NaN,NaN,Outflow,100.0,1.176471,0,NaN,NaN,NaN,7.206897
2,KELDK21,NaN,20sep2012,22oct2013,397,13,04=Diaries Interview,02oct2013,10,2013,...,NaN,NaN,NaN,0.0,0.000000,0,NaN,NaN,NaN,13.000000
3,KMOMT22,NaN,21sep2012,01nov2013,406,14,04=Diaries Interview,21feb2013,2,2013,...,NaN,NaN,Inflow,710.0,8.352942,0,NaN,NaN,NaN,4.793103
4,KMOMT32,NaN,20sep2012,18oct2013,393,13,04=Diaries Interview,02may2013,5,2013,...,NaN,NaN,Outflow,100.0,1.176471,0,NaN,NaN,NaN,6.517241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483944,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,02aug2013,8,2013,...,0.378316,NaN,Outflow,30.0,0.352941,0,NaN,NaN,NaN,4.586207
483945,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,04oct2013,10,2013,...,0.378316,NaN,Outflow,100.0,1.176471,0,NaN,NaN,NaN,6.448276
483946,KZIMA08,NaN,21mar2013,21oct2013,214,7,04=Diaries Interview,21aug2013,8,2013,...,0.378316,"08=Restaurant, bar, hotel, cyber",Outflow,50.0,0.588235,0,NaN,NaN,NaN,4.827586
483947,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,01aug2013,8,2013,...,74.572090,"10=Institution (clinic, school, church, etc.)",Outflow,1650.0,19.411760,0,NaN,NaN,NaN,10.965520


In [ ]:
def merge_datasets(left_df, right_df, key, second_key=None):

  if second_key==None:
    df_final=left_df.merge(df_dic[right_df], how="left", on=key)
  if second_key!=None:
    df_final=left_df.merge(df_dic[right_df], how="left", on_left=key, on_right=second_key)
  return df_final


In [ ]:
df_basic = df_dic['diaries_transactions_all']

dict_prova = {['hh_ids']: ['demo_hh', 'con_prod_hh_over_trx_yr_mo_usd', 'edu_hh'], 
              ['hh_ids', 'int_date']: ['diaries_wellbeing_ind', '' ]}
list_key_pairs = (                 ())
df= df_basic
for i in list_key_pairs:
  df=merge_datasets(df, right_df, key, second_key=None)

In [15]:
df_dic.keys()

dict_keys(['con_prod_hh_over_trx_yr_mo_usd', 'con_pur_hh_by_trx_yr_mo_usd', 'demo_hh', 'demo_ind', 'diaries_goingson_hh', 'diaries_wellbeing_ind', 'edu_hh', 'edu_ind', 'housing_conditions_ownership', 'inc_hh_by_trx_yr_mo_usd_incRR', 'inc_ind_by_trx_yr_mo_usd_incRR', 'pov_hh_by_trx_yr_mo_usd_incRR', 'rem_hh_by_trx_yr_mo_usd', 'pov_hh_by_trx_yr_mo_usd_excRR', 'diaries_transactions_all'])